In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [44]:
# Load the dataset
file_path = 'C:/Users/suhas/Downloads/df_iemocap_with_features.csv'  
df = pd.read_csv(file_path)


In [45]:
df_filtered = df[df['emotion'] != 'xxx'].reset_index(drop=True)

In [46]:
# Define features and label
X = df_filtered.drop(columns=['start_time', 'end_time', 'wav_file', 'emotion'])  # Drop non-feature columns
y = df_filtered['emotion']  # The target is the 'emotion' column

In [47]:
# Encode the emotions as numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [48]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) 


In [49]:
# Reshape the data for LSTM (samples, time steps, features)
X_lstm = np.expand_dims(X_scaled, axis=1)

In [50]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_encoded, test_size=0.2, random_state=42)


In [58]:
# Convert the labels to one-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)


In [59]:
num_classes = y_train_categorical.shape[1]
model = Sequential()
model.add(LSTM(128, input_shape=(1, 36), activation='relu', return_sequences=True))  # Larger LSTM
model.add(Dropout(0.4))  # Increased dropout
model.add(LSTM(64, activation='relu'))  # Second LSTM layer
model.add(BatchNormalization())  # Batch Normalization
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Output layer for multi-class classification


In [60]:
optimizer = Adam(learning_rate=0.0005)  # Adjusted learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [62]:
# Train the model
history = model.fit(X_train, y_train_categorical, epochs=50, batch_size=32, validation_data=(X_test, y_test_categorical))

Epoch 1/50
42/42 [==============================] - 9s 33ms/step - loss: 1.9264 - accuracy: 0.2682 - val_loss: 2.0097 - val_accuracy: 0.3576
Epoch 2/50
42/42 [==============================] - 0s 10ms/step - loss: 1.6329 - accuracy: 0.4439 - val_loss: 1.9253 - val_accuracy: 0.3636
Epoch 3/50
42/42 [==============================] - 0s 9ms/step - loss: 1.3884 - accuracy: 0.5136 - val_loss: 1.8491 - val_accuracy: 0.3697
Epoch 4/50
42/42 [==============================] - 0s 10ms/step - loss: 1.2385 - accuracy: 0.5470 - val_loss: 1.7689 - val_accuracy: 0.4515
Epoch 5/50
42/42 [==============================] - 0s 10ms/step - loss: 1.1546 - accuracy: 0.5667 - val_loss: 1.6877 - val_accuracy: 0.5091
Epoch 6/50
42/42 [==============================] - 0s 11ms/step - loss: 1.0607 - accuracy: 0.6038 - val_loss: 1.5985 - val_accuracy: 0.5485
Epoch 7/50
42/42 [==============================] - 0s 11ms/step - loss: 1.0162 - accuracy: 0.6174 - val_loss: 1.5177 - val_accuracy: 0.5636
Epoch 8/50
42/

In [63]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test_categorical)
print(f'Test Accuracy: {test_acc:.4f}')

11/11 [==============================] - 0s 5ms/step - loss: 1.1307 - accuracy: 0.6545
Test Accuracy: 0.6545


In [64]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 1, 128)            84480     
                                                                 
 dropout_5 (Dropout)         (None, 1, 128)            0         
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 batch_normalization_3 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 8)                 264       
                                                      